<a href="https://colab.research.google.com/github/alibaba35t/ChallengeWithAffectnet/blob/main/FacialEmotionRecognitionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
import os
# if avaliable, I prefer connect with Gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
#Download affectnet dataset for face emotion recognition
import kagglehub

path = kagglehub.dataset_download("mstjebashazida/affectnet")

print("Path to dataset files:", path)



100%|██████████| 324M/324M [00:02<00:00, 124MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mstjebashazida/affectnet/versions/1


In [3]:
# After installation of dataset, you can use this methods to see dataset folders with copying
import os
import shutil

visible_path = "/content"

# if you want you can delete existing files
if os.path.exists(visible_path):
    shutil.rmtree(visible_path)

# After installation of dataset, you can use this methods to see dataset folders with copying
shutil.copytree(path, visible_path)

print("Visible dataset path:", visible_path)
print("Folders:", os.listdir(visible_path))



Visible dataset path: /content
Folders: ['archive (3)']


cp: cannot stat '/content/drive/MyDrive/archive': No such file or directory


In [4]:
#We will create custom dataset for affectnet files because our labels were written different file format(.csv)
class AffectNetDataset(Dataset):
  def __init__(self,csv_path , image_dir , transform):
    super().__init__()
    self.csv_path = pd.read_csv(csv_path)
    self.image_dir = image_dir
    self.transform = transform
    self.label_map = {
        'surprise':0,
        'anger':1,
        'disgust':2,
        'fear':3,
        'sad':4,
        'contempt':5,
        'neutral':6,
        'happy':7
    }

  def __len__(self):
    return len(self.csv_path)

  def __getitem__(self, index) :
    row = self.csv_path.iloc[index]
    subset = row["subset"]
    img_path = os.path.join(self.image_dir, subset ,row ["pth"])
    # My subset script matched some files subset wrong, therefore I skipped rows when errors raised.
    # Don't worry I'll handle it. I am new in this industry.
    try:
      image = Image.open(img_path).convert("RGB")
      label = int(self.label_map[row["label"]])
      if self.transform:
         image = self.transform(image)
         return image, label
    except FileNotFoundError:
      index = (index + 1) % len(self.csv_path)





In [5]:
# Our transform codes, I made some kind of transform operations to make analysis easily
transform = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=1),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.RandomHorizontalFlip(),  # yatay çevirme
        transforms.RandomRotation(10),
        transforms.Resize((48,48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
    ])

In [10]:
# Paths for dataset files
csv_path = "/content/archive (3)/new_labels_V2.csv"
img_dir = "/content/archive (3)"
# Skip if it is None in batch
def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    return torch.utils.data.dataloader.default_collate(batch)
# Custom datasets splits
dataset = AffectNetDataset(csv_path=csv_path, image_dir=img_dir, transform=transform)
train_len = int(len(dataset)*0.8)
test_len = len(dataset) - train_len
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_len,test_len])
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, collate_fn=collate_skip_none)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, collate_fn=collate_skip_none)



In [7]:
from torch.nn.modules.pooling import MaxPool2d
# My model
from torch.nn.modules import ReLU

class FERmodel(nn.Module):
  def __init__(self):
    super(FERmodel,self).__init__()
    self.conv_layers = nn.Sequential(
         nn.Conv2d(1,32,kernel_size=3, padding=1),
         nn.BatchNorm2d(32),
         nn.ReLU(),
         nn.MaxPool2d(2,2),

         nn.Conv2d(32,64,kernel_size=3, padding=1),
         nn.BatchNorm2d(64),
         nn.ReLU(),


         nn.Conv2d(64,64,kernel_size=3, padding=1),
         nn.BatchNorm2d(64),
         nn.ReLU(),
         #nn.MaxPool2d(2,2),

         nn.Conv2d(64,128,kernel_size=3, padding=1),
         nn.BatchNorm2d(128),
         nn.ReLU(),
         nn.MaxPool2d(2,2),

         nn.Conv2d(128,256,kernel_size=3, padding=1),
         nn.BatchNorm2d(256),
         nn.ReLU(),
         #nn.MaxPool2d(2,2),
         nn.Conv2d(256,256,kernel_size=3, padding=1),
         nn.BatchNorm2d(256),
         nn.ReLU(),

         nn.Conv2d(256,512,kernel_size=3, padding=1),
         nn.BatchNorm2d(512),
         nn.ReLU(),
         nn.MaxPool2d(2,2)
     )
    self.fc_layers = nn.Sequential(
        nn.Linear(512*6*6,512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512,8),
    )
  def forward(self,x):
    x = self.conv_layers(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layers(x)
    return x

In [8]:
model = FERmodel()
model.load_state_dict(torch.load("/content/FERmodel_NewARCH_epoch11.pt"))
model.eval()

FERmodel(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), paddin

In [11]:


# You can choose other loss functions and optimizers because I am experimenting them now
criterion = nn.CrossEntropyLoss(label_smoothing=0.11) # label smoothing added
optimizer = optim.Adamax(params=model.parameters(), lr=0.00001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min', factor=0.5, patience=3)

#Training and evaluating process
def model_run(model,train_loader, test_loader,epochs):
  new_epoch = 12
  for new_epoch in range(epochs):
    model.train()
    val_los= 0.0
    for images ,labels in train_loader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      val_los += loss.item() * images.size(0)

    train_loss = val_los / len(train_loader.dataset)


    total = 0
    correct = 0
    model.eval()
    with torch.no_grad():
      for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    scheduler.step(val_los)
    print(f"Epoch [{new_epoch+1}/{epochs}]  Train Loss: {train_loss:.4f}  Test Accuracy: {accuracy:.2f}%")
    torch.save(model.state_dict(), f"/content/FERmodel_NewARCH_epoch{new_epoch}.pt")

model_run(model=model, train_loader=train_loader,test_loader=test_loader, epochs=50)

Epoch [1/50]  Train Loss: 1.2082  Test Accuracy: 71.25%
Epoch [2/50]  Train Loss: 1.1915  Test Accuracy: 71.59%
Epoch [3/50]  Train Loss: 1.1803  Test Accuracy: 71.46%
Epoch [4/50]  Train Loss: 1.1700  Test Accuracy: 72.14%
Epoch [5/50]  Train Loss: 1.1593  Test Accuracy: 71.38%
Epoch [6/50]  Train Loss: 1.1594  Test Accuracy: 71.73%


KeyboardInterrupt: 